In [43]:
import pandas as pd
import numpy as np
df = pd.read_csv('taxi.csv', index_col = 'Unnamed: 0')

In [44]:
df.head()

,count(1),month,year,dayofweek,hour,pickup_community_area
0,2742,11.0,2014.0,7.0,21.0,32.0
1,200,11.0,2014.0,6.0,23.0,77.0
2,110,4.0,2014.0,7.0,19.0,16.0
3,3205,12.0,2016.0,4.0,16.0,8.0
4,1159,12.0,2014.0,7.0,22.0,24.0


In [45]:
df.size

2876160

In [46]:
df.isnull().values.any()

True

In [47]:
df.isnull().sum()

count(1)                    0
month                       1
year                        1
dayofweek                   1
hour                        1
pickup_community_area    9242
dtype: int64

In [48]:
df.rename(columns={'count(1)':'count'}, inplace=True)


In [49]:
nan_rows = df[df.isnull().T.any().T]
nan_rows.head()

,count,month,year,dayofweek,hour,pickup_community_area
351,3019,12.0,2013.0,5.0,21.0,NaN
356,4253,5.0,2013.0,4.0,19.0,NaN
377,2545,10.0,2015.0,4.0,22.0,NaN
379,5670,6.0,2013.0,1.0,0.0,NaN
410,1835,6.0,2016.0,2.0,22.0,NaN


In [50]:
df=df.dropna(subset=['pickup_community_area'])
df.size

2820708

In [51]:
df.year = df.year.astype(int)
df.month = df.month.astype(int)
df.hour = df.hour.astype(int)
df.dayofweek = df.dayofweek.astype(int)
df.pickup_community_area = df.pickup_community_area.astype(int)
df.head()

,count,month,year,dayofweek,hour,pickup_community_area
0,2742,11,2014,7,21,32
1,200,11,2014,6,23,77
2,110,4,2014,7,19,16
3,3205,12,2016,4,16,8
4,1159,12,2014,7,22,24


In [52]:
df=df[df.year == 2014]
df=df[df.dayofweek == 7]
df=df[df.hour == 18]
df=df[df.month == 11]
df.head()

,count,month,year,dayofweek,hour,pickup_community_area
7179,1,11,2014,7,18,18
13993,2,11,2014,7,18,59
48006,4024,11,2014,7,18,32
48298,1066,11,2014,7,18,24
87560,12,11,2014,7,18,17


In [53]:
df.size

324

In [ ]:
#df= df.drop(["month", "year", "dayofweek", "hour"], axis=1)

In [60]:
df.columns

Index(['count', 'pickup_community_area'], dtype='object')

In [62]:
df.head()

,count,pickup_community_area
7179,1,18
13993,2,59
48006,4024,32
48298,1066,24
87560,12,17


In [71]:
df.to_csv("out.csv", index=False)

In [72]:
import csv


def main():
    # Dictionary to store the count of each community
    count_dict = {}
    # Open the file
    with open('out.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        # Count the number of rows
        num_rows = 0
        for row in csv_reader:
            # If it's the first row, do nothing
            if num_rows != 0:
                count_dict[int(row[1])] = int(row[0])
            num_rows += 1
        # for each key in the dict

        print(0, end="\t")
        for key in sorted(count_dict.keys()):
            print(key, end="\t")
        print()
        # For each key
        for key in sorted(count_dict.keys()):
            # Print the key
            print(key, end="\t")
            # get all the other keys
            for key2 in sorted(count_dict.keys()):
                # Print the difference
                print(count_dict[key] - count_dict[key2], end="\t")
            # print an empty line
            print()
main()

0	1	2	3	4	5	6	7	8	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	27	28	29	30	31	32	33	34	35	37	38	39	41	42	43	44	45	46	49	56	58	59	60	61	65	67	68	69	76	77	
1	0	-38	-398	-88	-215	-1970	-1824	-7283	50	41	52	51	15	49	-33	49	60	56	60	-1	-343	56	-1005	59	60	-1541	55	58	44	-3963	-719	32	38	59	55	42	-27	55	58	60	59	60	60	-223	58	59	53	58	60	60	59	57	-1017	-193	
2	38	0	-360	-50	-177	-1932	-1786	-7245	88	79	90	89	53	87	5	87	98	94	98	37	-305	94	-967	97	98	-1503	93	96	82	-3925	-681	70	76	97	93	80	11	93	96	98	97	98	98	-185	96	97	91	96	98	98	97	95	-979	-155	
3	398	360	0	310	183	-1572	-1426	-6885	448	439	450	449	413	447	365	447	458	454	458	397	55	454	-607	457	458	-1143	453	456	442	-3565	-321	430	436	457	453	440	371	453	456	458	457	458	458	175	456	457	451	456	458	458	457	455	-619	205	
4	88	50	-310	0	-127	-1882	-1736	-7195	138	129	140	139	103	137	55	137	148	144	148	87	-255	144	-917	147	148	-1453	143	146	132	-3875	-631	120	126	147	143	130	61	143	146	148	147	148	148	-135	146	147	141	146	148	148	147	145

76	1017	979	619	929	802	-953	-807	-6266	1067	1058	1069	1068	1032	1066	984	1066	1077	1073	1077	1016	674	1073	12	1076	1077	-524	1072	1075	1061	-2946	298	1049	1055	1076	1072	1059	990	1072	1075	1077	1076	1077	1077	794	1075	1076	1070	1075	1077	1077	1076	1074	0	824	
77	193	155	-205	105	-22	-1777	-1631	-7090	243	234	245	244	208	242	160	242	253	249	253	192	-150	249	-812	252	253	-1348	248	251	237	-3770	-526	225	231	252	248	235	166	248	251	253	252	253	253	-30	251	252	246	251	253	253	252	250	-824	0	
